In [1]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2

import pandas as pd

In [2]:
df = pd.read_csv('data/document_ratings.csv')
df.head()

,PY,content,id,relevant,seen,title,wosarticle__de,wosarticle__wc,1 - Adaptation,1 - Mitigation,...,mit/ad,1 - Mitigation - prediction,1 - Adaptation - prediction,3 - Economic instruments - prediction,3 - Goal setting - prediction,3 - Information policies - prediction,3 - International policies - prediction,3 - Planning and public provisioning/services - prediction,3 - Regulatory instruments - prediction,3 - Voluntary actions - prediction
0,1992.0,Some new aspects of microbiology in water-in-o...,599544,0.0,0.0,SOLUBILIZATION AND GROWTH OF CANDIDA-PSEUDOTRO...,YEAST; GROWTH; ORGANIC SOLVENT; MICROEMULSION;...,['Biotechnology & Applied Microbiology'],0.0,0.0,...,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018.0,The present experimental study is investigated...,1305496,0.0,0.0,Novel Garcinia gummi-gutta methyl ester (GGME)...,Garcinia gummi-gutta methyl ester; Immobilized...,['Green & Sustainable Science & Technology; En...,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2000.0,The DAYCENT ecosystem model was used to addres...,1860140,0.0,0.0,Interaction of soil carbon sequestration and N...,soil C; N2O flux; climate forcing; land use ma...,['Environmental Sciences'],0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1997.0,A new air mass-based synoptic procedure is use...,615984,0.0,0.0,An evaluation of climate/mortality relationshi...,acclimatization; air mass; climate and human m...,"['Environmental Sciences; Public, Environmenta...",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2009.0,Fresh produce is increasingly implicated in fo...,328444,0.0,0.0,Persistence of Escherichia coli on Injured ice...,0,['Biotechnology & Applied Microbiology; Food S...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from mordecai import Geoparser
geo = Geoparser()

geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")

[{'country_conf': 0.611725,
  'country_predicted': 'GBR',
  'geo': {'admin1': 'England',
   'country_code3': 'GBR',
   'feature_class': 'S',
   'feature_code': 'BDG',
   'geonameid': '6619889',
   'lat': '51.50821',
   'lon': '-0.08763',
   'place_name': 'London Bridge'},
  'spans': [{'end': 51, 'start': 38}],
  'word': 'London Bridge'}]

In [5]:
places = []
geos = []

for i, row in df.iterrows():
    
    t = row['title'] + row['content']
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \(C\) Copyright",t)[0]     
    
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            p["geo"]["doc_id"] = row['id']
            p["geo"]["word"] = p["word"]
            p["geo"]["country_predicted"] = p["country_predicted"]
            p["geo"]["country_conf"] = p["country_conf"]
            geos.append(p["geo"])
        rplaces.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)

In [7]:
geo_df = pd.DataFrame.from_dict(geos)
geo_df.to_csv('data/geo_df.csv', index=False)
geo_df.head()

,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf
0,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
1,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
2,Oklahoma,35.44951,-97.3967,USA,4542765,Midwest City,P,PPL,615984,Midwest,USA,0.999811
3,NA,39.76,-98.5,USA,6252001,United States,A,PCLI,615984,United States,USA,0.999811
4,West Virginia,39.23942,-80.48367,USA,4822880,Southwest District,A,ADMD,615984,Southwest,USA,0.939277


In [10]:
geo_df.head(100)

,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,word,country_predicted,country_conf
0,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
1,Aquitaine-Limousin-Poitou-Charentes,44.69935,1.24017,FRA,3018220,Florimont-Gaumier,P,PPL,1305496,GGME,FRA,0.731971
2,Oklahoma,35.44951,-97.3967,USA,4542765,Midwest City,P,PPL,615984,Midwest,USA,0.999811
3,NA,39.76,-98.5,USA,6252001,United States,A,PCLI,615984,United States,USA,0.999811
4,West Virginia,39.23942,-80.48367,USA,4822880,Southwest District,A,ADMD,615984,Southwest,USA,0.939277
...,...,...,...,...,...,...,...,...,...,...,...,...
95,NA,15.5,-90.25,GTM,3595528,Republic of Guatemala,A,PCLI,1296156,Guatemala,GTM,0.999931
96,Petén,16.99571,-89.81366,GTM,3591406,Lake Petén Itzá,H,LK,1296156,Lake Peten Itza,GTM,0.999931
97,NA,15.5,-90.25,GTM,3595528,Republic of Guatemala,A,PCLI,1296156,Guatemala,GTM,0.999931
98,Petén,16.99571,-89.81366,GTM,3591406,Lake Petén Itzá,H,LK,1296156,Peten Itza,GTM,0.999931
